In [1]:
import pandas as pd
import numpy as np

In [2]:
games_data = pd.read_csv("ginf.csv")
game_events = pd.read_csv("events.csv")
games_data = games_data[games_data['adv_stats'] == True]

In [3]:
games_data.drop(games_data.columns[-4:], axis=1, inplace=True)
def f(x):
    if x['fthg'] > x['ftag']: return 1
    else: return 0
games_data['class'] = games_data.apply(f, axis=1)

In [4]:
games_data = pd.concat([games_data, pd.get_dummies(games_data.league).rename(columns = "{}_league_binary".format)], axis = 1)
games_data = pd.concat([games_data, pd.get_dummies(games_data.country).rename(columns = "{}_country_binary".format)], axis = 1)
games_data.drop(["link_odsp","date","adv_stats","league","ht","at","country"], axis=1, inplace=True)
games_data.set_index('id_odsp',inplace = True)

In [5]:
game_events_m = game_events.copy()
event_type1 = {1:"Attempt(shot)", 2:"Corner", 3:"Foul", 4:"YellowCard", 5:"SecondYellowCard", 6:"StraightRedCard", 7:"Substitution", 8:"FreeKickWon", 9:"Offside", 10:"HandBall", 11:"PenaltyConceded"}
event_type2 = {12:"KeyPass", 13:"FailedThroughBall", 14:"SendingOff", 15:"OwnGoal"}
shot_outcome = {1:"OnTarget", 2:"OffTarget", 3:"Blocked", 4:"HitThePost"}
assist_method = {0:"None",1:"Pass",2:"Cross", 3: "HeadedPass", 4: "ThroughBall"}
situation = {1:"OpenPlay",2:"SetPiece",3:"Corner",4:"FreeKick"}
game_events_m.replace({"event_type": event_type1,"event_type2":event_type2,"side":{2:0,1:1}, "shot_outcome": shot_outcome,"assist_method":assist_method, "situation":situation}, inplace = True)
game_events_m.rename(columns={"side": "home_event"}, inplace = True)
game_events_m = pd.concat([game_events_m, pd.get_dummies(game_events_m.event_type).rename(columns = "{}_EventType_binary".format)], axis = 1)
game_events_m = pd.concat([game_events_m, pd.get_dummies(game_events_m.event_type2).rename(columns = "{}_EventType2_binary".format)], axis = 1)
game_events_m = pd.concat([game_events_m, pd.get_dummies(game_events_m.shot_outcome).rename(columns = "{}_ShotOutcome_binary".format)], axis = 1)
game_events_m = pd.concat([game_events_m, pd.get_dummies(game_events_m.assist_method).rename(columns = "{}_AssistMethod_binary".format)], axis = 1)
game_events_m = pd.concat([game_events_m, pd.get_dummies(game_events_m.situation).rename(columns = "{}_Situation_binary".format)], axis = 1)
game_events_m.drop(["text","event_team","opponent","is_goal","player_in","player_out", "shot_place","location","bodypart", "player","player2","event_type","event_type2","shot_outcome","assist_method","situation"], axis=1, inplace=True)

In [6]:
event_columns = game_events_m.columns.tolist()
event_columns.remove('id_odsp')
event_columns.remove('id_event')
event_columns.remove('sort_order')
all_games = games_data.index.tolist()
dev_games = pd.read_csv("dev_data_aggregate.csv")['id_odsp'].tolist()
nondev_games = list(set(all_games) - set(dev_games))
game_event_dt = game_events_m[game_events_m['sort_order'] < 101]
game_event_dt=game_event_dt[game_event_dt['id_odsp'].isin(nondev_games)]

In [7]:
for i in range(1,101):
    for column in event_columns:
        games_data[str(i)+"-"+column] = ""
for index, row in game_event_dt.iterrows(): 
    order = str(row['sort_order'])
    event_id = row['id_odsp']
    for column1 in event_columns:
        column_name = order+"-"+column1
        if str(row[column1]) != "":
            games_data.at[event_id, column_name] = row[column1]

C:\Users\yingy\AppData\Local\Temp/ipykernel_7124/2688329987.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  games_data[str(i)+"-"+column] = ""


In [27]:
columns = games_data.columns
columns_other = games_data.columns[0:18]
columns = [x for x in columns if "-" in x]
aggregate_events = [1,11,21,31,41,51,61,71,81,91]
aggregate_dic = {}
for a in aggregate_events:
    raw_list = []
    for i in range(a,a+10):
        col = [x.split("-")[1] for x in columns if str(i)+"-" in x]
        raw_list.extend(col)
    raw_list= list(set(raw_list))
    aggregate_dic[a] = raw_list
def process_row (row,aggregate_dic):
    row_dict = {}
    for k in aggregate_dic.keys():
        columns = aggregate_dic[k]
        for i in range(k,k+10):
            for c in columns:
                c_name = str(k)+"_"+str(k+10)+"_"+c.replace("_binary","")
                if row[str(i)+"-"+c] != '':
                    if int(row[str(i)+"-"+c]) > 0: 
                        if c_name in row_dict.keys():
                            row_dict[c_name] = row_dict[c_name] + int(row[str(i)+"-"+c])
                        else:
                            row_dict[c_name] =int(row[str(i)+"-"+c])
    return row_dict

In [12]:
aggregate_dic

{1: ['FreeKick_Situation_binary',
  'OpenPlay_Situation_binary',
  'HandBall_EventType_binary',
  'FreeKickWon_EventType_binary',
  'ThroughBall_AssistMethod_binary',
  'Corner_Situation_binary',
  'HeadedPass_AssistMethod_binary',
  'fast_break',
  'Substitution_EventType_binary',
  'time',
  'SecondYellowCard_EventType_binary',
  'Pass_AssistMethod_binary',
  'home_event',
  'Cross_AssistMethod_binary',
  'Attempt(shot)_EventType_binary',
  'StraightRedCard_EventType_binary',
  'OffTarget_ShotOutcome_binary',
  'PenaltyConceded_EventType_binary',
  'Blocked_ShotOutcome_binary',
  'HitThePost_ShotOutcome_binary',
  'YellowCard_EventType_binary',
  'OwnGoal_EventType2_binary',
  'SendingOff_EventType2_binary',
  'KeyPass_EventType2_binary',
  'SetPiece_Situation_binary',
  'OnTarget_ShotOutcome_binary',
  'Offside_EventType_binary',
  'Foul_EventType_binary',
  'None_AssistMethod_binary',
  'Corner_EventType_binary',
  'FailedThroughBall_EventType2_binary'],
 11: ['FreeKick_Situation_b

In [28]:
df_new = pd.DataFrame()
for index, row in games_data.iterrows():
    row_dict = process_row(row,aggregate_dic)
    new_row = pd.DataFrame(row_dict, index=[index,])
    for c in columns_other:
        new_row[c] = row[c]
    df_new =  pd.concat([df_new, new_row])

In [30]:
df_new.to_csv("Nondev_data_aggregate1.csv", encoding = 'utf-8',index=True)